The other notebook is for creating the directories and such. This one is for the implementation of the model.

In [0]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
df = pd.read_csv('gdrive/My Drive/ML/Projects/Kaggle/Aptos/train.csv')
fname_dict = {0 : [], 1 : [], 2 : [], 3 : [], 4 : []}
for index, row in df.iterrows():
  fname_dict[row['diagnosis']] += [row['id_code']]

In [0]:
original_dataset_dir = 'gdrive/My Drive/ML/Projects/Kaggle/Aptos/train_images'


In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale = 1. / 255, rotation_range=40, width_shift_range=0.25, height_shift_range=0.25,
                            shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
train_datagen = datagen.flow_from_directory(original_dataset_dir, target_size=(800, 800), batch_size=20, class_mode='categorical')

Found 2869 images belonging to 5 classes.


In [0]:
from keras.applications import VGG16


In [0]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.SeparableConv2D(64, (5, 5), activation='relu',
                        input_shape=(800, 800, 3)))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.SeparableConv2D(128, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((3, 3)))
model.add(layers.SeparableConv2D(256, (4, 4), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(128, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(256, (3, 3), activation='elu'))
model.add(layers.SeparableConv2D(128, (3, 3), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(128, (1, 1), activation='elu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SeparableConv2D(128, (3, 3), activation='elu'))
#model.add(layers.GlobalMaxPooling1D())
model.add(layers.Flatten())
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='elu'))
model.add(layers.BatchNormalization())
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='elu'))
model.add(layers.Dense(5, activation='softmax'))

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 796, 796, 64)      4864      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 265, 265, 64)      0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 263, 263, 128)     8896      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 87, 87, 128)       0         
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 84, 84, 256)       35072     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 42, 42, 256)       0         
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 40, 40, 128)       35200     
__________

In [0]:
from keras import optimizers

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adagrad(), metrics=['acc'])#, decay=0.03), metrics=['acc'])

In [0]:
history = model.fit_generator(
      train_datagen,
      steps_per_epoch=50,
      epochs=100)
      #validation_data=validation_generator,
      #validation_steps=50)

Epoch 1/100
50/50 [==============================] - 267s 5s/step - loss: 1.3054 - acc: 0.5230
Epoch 2/100
50/50 [==============================] - 264s 5s/step - loss: 1.2109 - acc: 0.5290
Epoch 3/100
50/50 [==============================] - 263s 5s/step - loss: 1.1310 - acc: 0.5661
Epoch 4/100
50/50 [==============================] - 269s 5s/step - loss: 1.0601 - acc: 0.6010
Epoch 5/100
50/50 [==============================] - 258s 5s/step - loss: 0.9923 - acc: 0.6350
Epoch 6/100
50/50 [==============================] - 261s 5s/step - loss: 0.9206 - acc: 0.6673
Epoch 7/100
50/50 [==============================] - 266s 5s/step - loss: 0.9131 - acc: 0.6770
Epoch 8/100
50/50 [==============================] - 257s 5s/step - loss: 0.8874 - acc: 0.6860
Epoch 9/100
50/50 [==============================] - 255s 5s/step - loss: 0.8402 - acc: 0.7055
Epoch 10/100
50/50 [==============================] - 264s 5s/step - loss: 0.8480 - acc: 0.7030
Epoch 11/100
50/50 [=============================

In [0]:
model.save('aptos_model.h5')

Adam with learning rate of 1e-3 (or 1e-4) gets the highest accuracy of what's been tested so far. 
Adagrad, does similarly and converges quite well after many epochs of training

Increase layer size, use pre-trained nets, implement dropout and l2 reg, etc